In [1]:
## https://sal.aalto.fi/publications/pdf-files/eluu11_public.pdf
## SYMBOLIC CALCULATION
import sympy as sym
import numpy as np
from sympy import *
from sympy.physics.units import speed_of_light, meter, second
from sympy.physics.units import convert_to
from sympy.physics.units import ampere, gram, second, kilogram, newton, convert_to
import scipy.optimize

In [2]:
##DEFINING CONSTANTS
gravity = sym.Symbol('g', real=True) #gravitational acceleration
mass = sym.Symbol('m', real=True) #mass of quadcopter
Fg = sym.Matrix([0,0,gravity*mass]) #Global force of gravity

## Inertias
Ixx, Iyy, Izz = sym.symbols('I_xx I_yy I_zz', real=True)
I = sym.Matrix([[Ixx, 0, 0],
                [0, Iyy, 0],
                [0, 0, Izz]])

Im = sym.Symbol('I_m', real=True) # Motor+propellor Inertia

#Distance of rotors from CoG
l = sym.Symbol('l', real=True)
## Rotor characteristics
rotorLiftConstant = sym.Symbol('k', real=True)
rotorDragConstant = sym.Symbol('b', real=True)


In [3]:
## INITIAL VARIABLES
x,y,z = sym.symbols('x y z', real=True)
positions = sym.Matrix([x,y,z]) #X,Y,Z

dx,dy,dz = sym.symbols('v_x v_y v_z', real=True)
velocities = sym.Matrix([dx,dy,dz]) #dX, dY, dZ

phi,theta,psi = sym.symbols('phi theta psi', real=True)
angles = sym.Matrix([phi,theta,psi]) #Inertial Angles

Wx,Wy,Wz = sym.symbols('omega_phi omega_theta, omega_psi', real=True) #Angular velocities
angularVelocities = sym.Matrix([Wx, Wy, Wz])

Wm1, Wm2, Wm3, Wm4 = sym.symbols('W_m1, W_m2, W_m3, W_m4', real=True,positive=True)
rotorW = sym.Matrix([Wm1, Wm2, Wm3, Wm4]) #Initial angular spin rates
T1, T2, T3, T4 = sym.symbols('tau_m1, tau_m2, tau_m3, tau_m4', real=True)
motorTorques = sym.Matrix([T1,T2,T3,T4]) #Commanded Motor Torques

In [4]:
## Linear rotation matrix construction
def getRotationMatrices(angles):
  phi = angles[0]
  theta = angles[1]
  psi = angles[2]
  Rx = sym.Matrix([[1,            0,             0],
                  [0, sym.cos(phi), -sym.sin(phi)],
                  [0, sym.sin(phi),  sym.cos(phi)]])
  Ry = sym.Matrix([[ sym.cos(theta), 0, sym.sin(theta)],
                  [              0, 1,              0],
                  [-sym.sin(theta), 0, sym.cos(theta)]])
  Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                  [sym.sin(psi),  sym.cos(psi), 0],
                  [           0,             0, 1]])
  RtoG = Rz @ Ry @ Rx ## Rotate to Global


  ## Angular Rotation Matrix
  ex = sym.Matrix([[1], [0], [0]])
  ey = sym.Matrix([[0], [1], [0]])
  ez = sym.Matrix([[0], [0], [1]])
  RAtoB = sym.Matrix.hstack(ex, Rx.T @ ey, (Ry @ Rx).T @ ez)

  return RtoG, RAtoB


In [5]:
#### MOTOR HANDLING
def symSum(vector):
  sum = 0
  for i in vector:
    sum = sum + i
  return sum

def getRotorAccelerations(rotor_angular_velocity,commanded_motor_torques):
  b = rotorDragConstant
  dragComponent = rotor_angular_velocity.applyfunc(lambda x: x**2 * b)
  motorComponent = commanded_motor_torques
  dWm = (motorComponent - dragComponent) / Im
  return dWm #Rotor acceleration

def getRotorForces(rotor_angular_velocity, commanded_rotor_torques):
  Wm1 = rotor_angular_velocity[0]
  Wm2 = rotor_angular_velocity[1]
  Wm3 = rotor_angular_velocity[2]
  Wm4 = rotor_angular_velocity[3]

  k = rotorLiftConstant
  b = rotorDragConstant
  individualThrusts = rotorW.applyfunc(lambda x: x**2 * k)
  thrust = sym.simplify(symSum(individualThrusts))#Calculating lift from blades
  thrust = sym.Matrix([0,0,thrust])

  tau_phi = l*k*(-Wm2**2 + Wm4**2)
  tau_theta = l*k*(-Wm1**2 + Wm3**2)
  dWm = getRotorAccelerations(rotor_angular_velocity, commanded_rotor_torques) #Derivitives of rotor acceleration
  [dWm1,dWm2,dWm3,dWm4] = dWm
  Tm1 = -b*Wm1**2 - Im*dWm1
  Tm2 = b*Wm2**2 + Im*dWm2
  Tm3 = -b*Wm3**2 -Im*dWm3
  Tm4 = b*Wm4**2 + Im*dWm4
  tau_psi = sym.simplify(Tm1+Tm2+Tm3+Tm4)
  tau = sym.Matrix([tau_phi, tau_theta, tau_psi])
  return thrust, tau, dWm

print(getRotorForces(rotorW,motorTorques))

(Matrix([
[                                        0],
[                                        0],
[k*(W_m1**2 + W_m2**2 + W_m3**2 + W_m4**2)]]), Matrix([
[          k*l*(-W_m2**2 + W_m4**2)],
[          k*l*(-W_m1**2 + W_m3**2)],
[-tau_m1 + tau_m2 - tau_m3 + tau_m4]]), Matrix([
[(-W_m1**2*b + tau_m1)/I_m],
[(-W_m2**2*b + tau_m2)/I_m],
[(-W_m3**2*b + tau_m3)/I_m],
[(-W_m4**2*b + tau_m4)/I_m]]))


In [6]:
def f_true(positions, velocities, angles, angularVelocities, rotorW,commanded_motor_torques):
  ## Rotation Matrices
  RtoG, RAtoB = getRotationMatrices(angles)
  RtoB = RtoG.T
  RAtoG = sym.simplify(RAtoB.inv()) ## Rotate angular to global

  #### Rotor Handling
  ## Body frame
  thrust, tau, dWm = getRotorForces(rotorW,commanded_motor_torques)
  ## Converting to global frame
  globalForcesThrust = sym.simplify(RtoG @ thrust)
  ## Global frame
  globalLinearAcceleration = sym.simplify((Fg + globalForcesThrust)/mass) #+ Drag term  #####SYMPLIFY MASSIVELY SLOWS DOWN COMPUTATION. REMOVE IF TOO SLOW
  globalAngularAcceleration =  sym.simplify(RAtoG @ (I.inv() @ tau))


  return [velocities, globalLinearAcceleration, angularVelocities, globalAngularAcceleration, dWm]
derivitivesW = f_true(positions, velocities, angles, angularVelocities, rotorW, motorTorques)
display(sym.Matrix(derivitivesW))

Matrix([
[                                                                                                                                                                              v_x],
[                                                                                                                                                                              v_y],
[                                                                                                                                                                              v_z],
[                                                                                   k*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))*(W_m1**2 + W_m2**2 + W_m3**2 + W_m4**2)/m],
[                                                                                  -k*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi))*(W_m1**2 + W_m2**2 + W_m3**2 + W_m4**2)/m],
[                                                                                     

In [7]:
derivitivesW2 = sym.Matrix(derivitivesW)
k = 0.000196800   # Lift constant in newtons per radian**2
b = 9.892246947623543e-06 # torque per radian ^2
m = 0.6945 # Mass of the drone (kg)
g = -9.81  # Acceleration due to gravity (m/s^2)
l = 0.45/2. # Radius of rotor to COG
I_xx = 0.01349  # Moment of inertia about x-axis
I_yy = 0.01349  # Moment of inertia about y-axis
I_zz = .02681 # Moment of inertia about z-axis
I_m = 1.6817e-05 # Moment of inertia of the rotor
derivitivesW2 = derivitivesW2.subs([(psi,0),(theta,0),(dz, 0), (dy,0), (Wx,0), (Wy,0), (T2, T1), (T3,T1), (T4,T1), (Wm2, Wm1), (Wm3, Wm1), (Wm4, Wm1)
                                    ,(rotorLiftConstant,k),(rotorDragConstant,b),(mass, m), (gravity, g),(Ixx,I_xx), (Iyy, I_yy), (Izz,I_zz),(Im,I_m),])

In [8]:
## Solving for exact solution to problem 2
speed = 0.5
radius = 2
centripital_acceleration = speed**2 / radius
centripital_acceleration

## Yaw rate
circumference = 2*sym.pi*radius
time_per_loop = circumference/speed
yawRate = 2*pi/time_per_loop

desiredStateMatrix = sym.Matrix([speed,0,0, ## Velocity
                                  0,centripital_acceleration,0, ## Acceleration
                                  0,0,yawRate, ## Angular velocity
                                  0,0,0, ## Angular acceleration
                                  0,0,0,0 ## Motor Torque
                                  ])

sym.Matrix(derivitivesW2-desiredStateMatrix)

Matrix([
[                                           v_x - 0.5],
[                                                   0],
[                                                   0],
[                                                   0],
[       -0.00113347732181425*W_m1**2*sin(phi) - 0.125],
[         0.00113347732181425*W_m1**2*cos(phi) - 9.81],
[                                                   0],
[                                                   0],
[                                    omega_psi - 0.25],
[                                                   0],
[                                                   0],
[                                                   0],
[-0.588228991355387*W_m1**2 + 59463.6379853719*tau_m1],
[-0.588228991355387*W_m1**2 + 59463.6379853719*tau_m1],
[-0.588228991355387*W_m1**2 + 59463.6379853719*tau_m1],
[-0.588228991355387*W_m1**2 + 59463.6379853719*tau_m1]])

In [9]:
circlesolution = sym.solve(derivitivesW2-desiredStateMatrix)

In [10]:
display((circlesolution))


[{W_m1: 93.0348609175189,
  omega_psi: 0.250000000000000,
  phi: -0.0127414103580744,
  tau_m1: 0.0856221984935959,
  v_x: 0.500000000000000}]

In [11]:
derivitivesW3 = sym.Matrix(derivitivesW)

##INITIAL CONDITIONS
derivitivesW3 = derivitivesW3.subs([(phi,0),(psi,0),(theta,0),(dz, 0),(dx,0), (dy,0),(rotorLiftConstant,k),(rotorDragConstant,b),(mass, m), (gravity, g),(Ixx,I_xx), (Iyy, I_yy), (Izz,I_zz),(Im,I_m),])

In [12]:
## Desired state
desiredStateMatrix = sym.Matrix([0,0,0,
                                  0,0,0,
                                  0,0,0,
                                  0,0,0,
                                  0,0,0,0
                                  ])

sym.Matrix(derivitivesW3-desiredStateMatrix)

Matrix([
[                                                                                                                               0],
[                                                                                                                               0],
[                                                                                                                               0],
[                                                                                                                               0],
[                                                                                                                               0],
[0.000283369330453564*W_m1**2 + 0.000283369330453564*W_m2**2 + 0.000283369330453564*W_m3**2 + 0.000283369330453564*W_m4**2 - 9.81],
[                                                                                                                       omega_phi],
[                                                                  

In [13]:
hoverSolution = sym.solve(derivitivesW3-desiredStateMatrix)

In [15]:
display((hoverSolution))

[{W_m1: 93.0310849898583,
  W_m2: 93.0310849898583,
  W_m3: 93.0310849898583,
  W_m4: 93.0310849898583,
  omega_phi: 0.0,
  omega_psi: 0.0,
  omega_theta: 0.0,
  tau_m1: 0.0856152484823067,
  tau_m2: 0.0856152484823067,
  tau_m3: 0.0856152484823067,
  tau_m4: 0.0856152484823067}]